## From C code to Julia code: some bad and good practices

---- 
### C Code

```C
#include <stdio.h>
#include <stdlib.h>

int main()
{
    int A[100][100][100];
    int i,j,k,loop;
    for (loop=0; loop <1000; loop++){
        for (i=0; i<100; i++){
            for (j=0; j<100; j++){
                for (k=0; k<100; k++){
                    A[i][j][k] = loop;
                }
            }
        }
        printf("%d\n", A[99][99][99]);
    }
    return 0;
}
```

In [ ]:
run(`gcc test.c -O3 -o test2025`)

In [ ]:
@time run(`./test2025`)

----
### Julia Codes

**test1.jl**

In [ ]:
@time begin
    
    A = zeros(100,100,100)
    for loop = 1:1000
        for i = 1:100
            for j = 1:100
                for k = 1:100
                    A[i,j,k] = loop
                end
            end
        end
        println(A[100,100,100])
    end
    
end

##### Remark:
coding without using function is bad in Julia $\longrightarrow$ modify

----
**test2.jl**

In [ ]:
function test2()
    A = zeros(100,100,100)
    for loop = 1:1000
        for i = 1:100
            for j = 1:100
                for k = 1:100
                    A[i,j,k] = loop
                end
            end
        end
        println(A[100,100,100])
    end
end

@time test2();

##### Remark:
memory representation of matrix `A` in C $\neq$ memory representation in Julia (same of ForTran) $\longrightarrow$ modify

----
**test3.jl**

In [ ]:
function test3()
    A = zeros(100,100,100)
    for loop = 1:1000
        for k = 1:100
            for j = 1:100
                for i = 1:100
                    A[i,j,k] = loop
                end
            end
        end
        println(A[100,100,100])
    end
end

@time test3();

##### Remark:
bounds of the matrix are implicitely checked $\longrightarrow$ modify in relaxing it

----
**test4.jl**

In [ ]:
function test4()
    A = zeros(100,100,100)
    for loop = 1:1000
        for k = 1:100
            for j = 1:100
                for i = 1:100
                    @inbounds A[i,j,k] = loop
                end
            end
        end
        println(A[100,100,100])
    end
end

@time test4();

##### Remark:

In [ ]:
typeof( zeros(100,100,100) )

----
**test5.jl**

In [ ]:
function test5()
    A = zeros(Int64,100,100,100)
    for loop = 1:1000
        for k = 1:100
            for j = 1:100
                for i = 1:100
                    @inbounds A[i,j,k] = loop
                end
            end
        end
        println(A[100,100,100])
    end
end

@time test5();

##### Remark:
`int` in C is coded on 32 bits $\longrightarrow$ modify to `Int32`

----
**test6.jl**

In [ ]:
function test6()
    A = zeros(Int32,100,100,100)
    for loop::Int32 = 1:1000
        for k = 1:100
            for j = 1:100
                for i = 1:100
                    @inbounds A[i,j,k] = loop
                end
            end
        end
        println(A[100,100,100])
    end
end

@time test6();

##### Remark:
The matrix is not initialized to `0` in the C code $\longrightarrow$ modify

----
**test7.jl**

In [ ]:
function test7()
    A = Array{Int32}(undef,100,100,100)
    for loop::Int32 = 1:1000
        for k = 1:100
            for j = 1:100
                for i = 1:100
                    @inbounds A[i,j,k] = loop
                end
            end
        end
        println(A[100,100,100])
    end
end

@time test7();

##### Remark:
compact coding for nested loops in Julia $\longrightarrow$ modify

----
**test8.jl**

In [ ]:
function test8()
    A = Array{Int32}(undef,100,100,100)
    for loop::Int32 = 1:1000
        for k = 1:100, j = 1:100, i = 1:100
            @inbounds A[i,j,k] = loop
        end
        println(A[100,100,100])
    end
end

@time test8();

----
**benchmark with test8.jl**

In [ ]:
using BenchmarkTools

In [ ]:
@btime test8()

In [ ]:
@benchmark test8()